# Estudo acoes

### Bibilioteca

In [410]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import pickle
from datetime import datetime
import requests
from sklearn.model_selection import train_test_split


### Busca e tratamento de dados

In [374]:
nome_acoes = pd.read_csv('./dados/nomes_acoes.csv', sep=',', encoding='utf-8')

In [375]:

def buscaDadosHistoricos(acao):
    base_url = "https://statusinvest.com.br/acao/indicatorhistoricallist"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    form_data = {
        'codes[]': acao.upper(),  # Convert to uppercase to ensure consistency
        'time': '5',
        'byQuarter': 'true',
        'futureDate': 'true'
    }

    response = requests.post(base_url, data=form_data, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # Normalize keys to uppercase for comparison
        normalized_data_keys = {k.upper(): v for k, v in data['data'].items()}

        if acao.upper() not in normalized_data_keys:
            print(f"No data found for {acao}")
            return pd.DataFrame()

        reshaped_data = []

        # Access the data using the normalized key
        for indicator in normalized_data_keys[acao.upper()]:
            for rank_info in indicator['ranks']:
                if rank_info['rank_F'] == "ATUAL":
                    continue

                entry = {
                    'ticker': f"{acao.upper()}_{rank_info['rank']}",
                    indicator['key']: rank_info['value_F'].replace('-', '0').replace('-%', '0')
                }
                reshaped_data.append(entry)

        reshaped_df = pd.DataFrame(reshaped_data)
        final_df = reshaped_df.groupby('ticker').first().reset_index()
        final_df.replace(['-', '-%'], '0', inplace=True)

        for col in final_df.columns[1:]:
            final_df[col] = pd.to_numeric(final_df[col].str.replace('%', '').str.replace(',', '.').replace('', '0'), errors='coerce')

        return final_df

    else:
        print(f"Error accessing URL: {response.status_code}")
        return pd.DataFrame()

# Assuming 'tabela_diferencas' is a pandas DataFrame with valid ticker symbols in the 'ticker' column
all_data_frames = [buscaDadosHistoricos(ticker) for ticker in nome_acoes['ticker']]

# Ensure all DataFrames are valid and not empty before concatenation
valid_data_frames = [df for df in all_data_frames if not df.empty]

# Concatenate all the valid DataFrames into a single one
df = pd.concat(valid_data_frames, ignore_index=True)

# Display the combined DataFrame
print(df.head())
df.fillna(0, inplace='True')
df.to_csv('./dados/dados_historicos.csv')


KeyboardInterrupt: 

In [ ]:

def fetchYearlyAveragePrice(acao):
    base_url = "https://statusinvest.com.br/acao/tickerprice"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    form_data = {
        'ticker': acao,
        'type': '4',
        'currences[]': '1'
    }

    response = requests.post(base_url, data=form_data, headers=headers)

    if response.status_code == 200:
        data = response.json()

        if not data or 'prices' not in data[0]:
            print(f"No 'prices' key or empty data for {acao}. Response: {data}")
            return pd.DataFrame()

        prices_data = [price for item in data for price in item.get('prices', [])]
        if not prices_data:
            print(f"No prices data extracted for {acao}. Item examined: {data[0]}")
            return pd.DataFrame()
        
        df = pd.DataFrame(prices_data)
        if 'date' not in df.columns or 'price' not in df.columns:
            print(f"Expected columns not found in data for {acao}. Columns: {df.columns}")
            return pd.DataFrame()

        df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y %H:%M')
        df['year'] = df['date'].dt.year
        df['ticker'] = acao.upper()

        yearly_avg_df = df.groupby(['ticker', 'year'])['price'].mean().reset_index()
        yearly_avg_df.rename(columns={'price': 'resultado'}, inplace=True)

        return yearly_avg_df
    else:
        print(f"Error accessing URL: {response.status_code}")
        return pd.DataFrame()

# Replace 'nome_acoes' with your DataFrame that contains the ticker list
all_data_frames = [fetchYearlyAveragePrice(ticker) for ticker in nome_acoes['ticker']]

combined_df_prices = pd.concat(all_data_frames, ignore_index=True)
print(combined_df_prices.head())
combined_df_prices.to_csv('./dados/precos_ano.csv')


No prices data extracted for MSRO3. Item examined: {'currencyType': 1, 'currency': 'Real brasileiro', 'symbol': 'R$', 'prices': []}
No prices data extracted for GUAR4. Item examined: {'currencyType': 1, 'currency': 'Real brasileiro', 'symbol': 'R$', 'prices': []}
No prices data extracted for MMAQ3. Item examined: {'currencyType': 1, 'currency': 'Real brasileiro', 'symbol': 'R$', 'prices': []}
No prices data extracted for CTSA8. Item examined: {'currencyType': 1, 'currency': 'Real brasileiro', 'symbol': 'R$', 'prices': []}
No prices data extracted for BPAT33. Item examined: {'currencyType': 1, 'currency': 'Real brasileiro', 'symbol': 'R$', 'prices': []}
No prices data extracted for LHER3. Item examined: {'currencyType': 1, 'currency': 'Real brasileiro', 'symbol': 'R$', 'prices': []}
No prices data extracted for LHER4. Item examined: {'currencyType': 1, 'currency': 'Real brasileiro', 'symbol': 'R$', 'prices': []}
No prices data extracted for CATA4. Item examined: {'currencyType': 1, 'cur

In [ ]:
combined_df_prices.columns

Index(['ticker', 'year', 'resultado'], dtype='object')

In [ ]:
df.ticker

0       MSRO3_2012
1       MSRO3_2013
2       MSRO3_2014
3       MSRO3_2015
4       MSRO3_2016
           ...    
8545    BRIT3_2019
8546    BRIT3_2020
8547    BRIT3_2021
8548    BRIT3_2022
8549    BRIT3_2023
Name: ticker, Length: 8550, dtype: object

In [ ]:
combined_df_prices['ticker_year'] = combined_df_prices['ticker'] + '_' + combined_df_prices['year'].astype(str)

# Passo 2: Realizar o merge
# Como você quer manter todos os registros de combined_df_prices, você fará um left join neste DataFrame
# Isto significa que a chave para o merge é 'ticker_year' de combined_df_prices e 'ticker' de df
merged_df = pd.merge(combined_df_prices, df, left_on='ticker_year', right_on='ticker', how='left')

# Se você quer adicionar a coluna 'price' em df, você pode considerar que o merge pode ser feito de outra forma
# dependendo de qual informação exata você quer preservar. Se df deve manter todas as linhas e receber informações de price onde aplicável, o join seria:

df = pd.merge(df, combined_df_prices[['ticker_year', 'resultado']], left_on='ticker', right_on='ticker_year', how='left')
df.drop('ticker_year', axis=1, inplace=True)  # Remover a coluna auxiliar

print(df)

      Unnamed: 0      ticker    dy     p_l  p_vp  p_ebita  p_ebit  p_sr   
0              0  MSRO3_2012  0.00    0.00  0.00     0.00    0.00  0.00  \
1              1  MSRO3_2013  0.00    0.00  0.00     0.00    0.00  0.00   
2              2  MSRO3_2014  0.00    0.00  0.00     0.00    0.00  0.00   
3              3  MSRO3_2015  0.00    0.00  0.00     0.00    0.00  0.00   
4              4  MSRO3_2016  0.00    0.00  0.00     0.00    0.00  0.00   
...          ...         ...   ...     ...   ...      ...     ...   ...   
8545        8545  BRIT3_2019  0.00    0.00  0.00     0.00    0.00  0.00   
8546        8546  BRIT3_2020  0.00    0.00  0.00     0.00    0.00  0.00   
8547        8547  BRIT3_2021  0.00  995.54  1.66     9.40   27.96  3.06   
8548        8548  BRIT3_2022  0.04   20.20  0.89     2.81    7.36  1.24   
8549        8549  BRIT3_2023  0.87   12.67  1.13     2.94    6.63  1.41   

      p_ativo  p_capitlgiro  ...  margemebitda  margemebit  margemliquida   
0        0.00         

In [ ]:
df.to_csv('./dados/dados_treino.csv')

In [ ]:
df2 = df.query('resultado>0')
df2.to_csv('./dados/dados_treino_filtrada.csv')

In [ ]:
df = pd.read_csv('./dados/dados_historicos.csv', sep=',', encoding='utf-8')

In [ ]:
df.head(4)

,Unnamed: 0,ticker,dy,p_l,p_vp,p_ebita,p_ebit,p_sr,p_ativo,p_capitlgiro,...,margembruta,margemebitda,margemebit,margemliquida,roe,roa,roic,giro_ativos,receitas_cagr5,lucros_cagr5
0,0,MSRO3_2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.21,19.00,4.46,6.86,12.56,3.33,4.40,0.49,0.0,0.0
1,1,MSRO3_2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.26,25.83,11.81,2.59,5.90,1.52,6.29,0.59,0.0,0.0
2,2,MSRO3_2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33.48,0.00,20.37,4.32,4.19,1.91,8.48,0.44,0.0,0.0
3,3,MSRO3_2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.76,22.31,10.11,15.16,25.30,8.62,2.43,0.57,0.0,0.0


In [ ]:
data_atual = datetime.now()

data_formatada = data_atual.strftime("%d_%m_%Y")


In [ ]:
df2.isna().sum()

Unnamed: 0                         0
ticker                             0
dy                                 0
p_l                                0
p_vp                               0
p_ebita                            0
p_ebit                             0
p_sr                               0
p_ativo                            0
p_capitlgiro                       0
p_ativocirculante                  0
ev_ebitda                          0
ev_ebit                            0
lpa                                0
vpa                                0
peg_Ratio                          0
dividaliquida_patrimonioliquido    0
dividaliquida_ebitda               0
dividaliquida_ebit                 0
patrimonio_ativo                   0
passivo_ativo                      0
liquidezcorrente                   0
margembruta                        0
margemebitda                       0
margemebit                         0
margemliquida                      0
roe                                0
r

### Definição alvo e previsores

In [387]:
df2 = pd.read_csv('./dados/dados_treino_filtrada.csv', sep=',', encoding='utf-8')

In [388]:
alvo = df2.resultado

In [398]:
previsores = df2[[ 'dy', 'p_l', 'p_vp', 'p_ebita', 'p_ebit', 'p_sr',
       'p_ativo', 'p_capitlgiro', 'p_ativocirculante', 'ev_ebitda', 'ev_ebit',
       'lpa', 'vpa', 'peg_Ratio', 'dividaliquida_patrimonioliquido',
       'dividaliquida_ebitda', 'dividaliquida_ebit', 'patrimonio_ativo',
       'passivo_ativo', 'liquidezcorrente', 'margembruta', 'margemebitda',
       'margemebit', 'margemliquida', 'roe', 'roa', 'roic', 'giro_ativos',
       'receitas_cagr5', 'lucros_cagr5']]

In [399]:
alvo.head(2)

0    16.871786
1     7.844340
Name: resultado, dtype: float64

In [400]:
previsores.head(3)

,dy,p_l,p_vp,p_ebita,p_ebit,p_sr,p_ativo,p_capitlgiro,p_ativocirculante,ev_ebitda,...,margembruta,margemebitda,margemebit,margemliquida,roe,roa,roic,giro_ativos,receitas_cagr5,lucros_cagr5
0,0.01,33.58,1.63,49.13,79.52,4.95,1.52,1.99,8.74,24.72,...,89.47,10.08,6.23,14.75,4.84,4.52,0.25,0.31,0.00,0.0
1,0.00,32.29,1.56,47.23,76.45,4.76,1.46,1.91,8.40,22.82,...,89.47,10.08,6.23,14.75,4.84,4.52,0.25,0.31,0.00,0.0
2,3.38,11.63,1.87,11.60,10.57,4.18,879.67,8.00,1.11,10.28,...,44.81,36.05,39.59,35.97,16.08,7.56,13.88,0.21,10.14,0.0


In [401]:
print(previsores.shape)
print(alvo.shape)

(2401, 30)
(2401,)


### Escalonamento

In [402]:
scaler = StandardScaler()
previsores_standard_scaling = scaler.fit_transform(previsores)

# Aplicando MinMaxScaler
min_max_scaler = MinMaxScaler()
previsores_min_scaling = min_max_scaler.fit_transform(previsores)

# Criando a pasta 'variaveis' se não existir
if not os.path.exists('variaveis'):
    os.makedirs('variaveis')

# Salvando os previsores escalonados
with open('variaveis/previsores_standard_scaling.pickle', 'wb') as f:
    pickle.dump(previsores_standard_scaling, f)

with open('variaveis/previsores_min_scaling.pickle', 'wb') as f:
    pickle.dump(previsores_min_scaling, f)

with open('variaveis/alvo.pickle', 'wb') as f:
    pickle.dump(alvo, f)

with open('variaveis/previsores.pickle', 'wb') as f:
    pickle.dump(previsores, f)

### Looping de algoritimos

In [403]:
previsor_escolhido = 'previsores'
with open(f'./variaveis/alvo.pickle', 'rb') as file:
    alvo = pickle.load(file)
with open(f'./variaveis/{previsor_escolhido}.pickle', 'rb') as file:
    previsores = pickle.load(file)

In [411]:
inicio_treinamento = datetime.now()

# Diretório para salvar os modelos treinados
algoritmos_dir = "./modelos"

# Dicionário de configuração dos modelos e parâmetros para GridSearch
modelos_config = {
    "random_forest": {
        "modelo": RandomForestRegressor(),
        "parametros": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10],
            "min_samples_split": [2, 5],
            "min_samples_leaf": [1, 2]
        }
    },
    # Adicione configurações similares para outros modelos se necessário
}

resultados = {}

for nome, config in modelos_config.items():
    print(f"Otimizando {nome}...")
    grid_search = GridSearchCV(estimator=config["modelo"], param_grid=config["parametros"], scoring='r2', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    resultados[nome] = {
        "best_params": grid_search.best_params_,
        "cv_r2": grid_search.best_score_,
        "r2": r2,
        "rmse": rmse,
    }

    # Salvando o melhor modelo
    with open(f'{algoritmos_dir}/{nome}_best_model.pickle', 'wb') as file:
        pickle.dump(best_model, file)

fim_treinamento = datetime.now()

# Consolidando e salvando os resultados
resultados_completos = {
    "resultados": resultados,
    "inicio": inicio_treinamento.strftime('%Y-%m-%d %H:%M:%S'),
    "fim": fim_treinamento.strftime('%Y-%m-%d %H:%M:%S')
}

with open(f'{algoritmos_dir}/resultado_completo_df.pickle', 'wb') as file:
    pickle.dump(resultados_completos, file)


Otimizando random_forest...


## Resultado

In [407]:
with open('./modelos/resultado_completo_df.pickle', 'rb') as file:
    resultados_completos = pickle.load(file)

# Imprimir os resultados de R^2 para referência
for nome, resultado in resultados_completos['resultados'].items():
    print(f"{nome}: R^2 no teste = {resultado['r2']}")

random_forest: R^2 no teste = -0.437320923167984
decision_tree: R^2 no teste = -0.9317690094105067
linear_regression: R^2 no teste = -0.12834681955125204
catboost: R^2 no teste = 0.14917045977240162
mlp: R^2 no teste = -0.013337116715886221
adaboost: R^2 no teste = -2.229719461726198
gradient_boosting: R^2 no teste = 0.16420010351063696


In [242]:
# URL base da API
base_url = "https://statusinvest.com.br/category/advancedsearchresultpaginated"

# Parâmetros da busca
search_params = {
    "search": '{"Sector":"","SubSector":"","Segment":"","my_range":"-20;100","forecast":{"upsidedownside":{"Item1":null,"Item2":null},"estimatesnumber":{"Item1":null,"Item2":null},"revisedup":true,"reviseddown":true,"consensus":[]},"dy":{"Item1":null,"Item2":null},"p_l":{"Item1":null,"Item2":null},"peg_ratio":{"Item1":null,"Item2":null},"p_vp":{"Item1":null,"Item2":null},"p_ativo":{"Item1":null,"Item2":null},"margembruta":{"Item1":null,"Item2":null},"margemebit":{"Item1":null,"Item2":null},"margemliquida":{"Item1":null,"Item2":null},"p_ebit":{"Item1":null,"Item2":null},"ev_ebit":{"Item1":null,"Item2":null},"dividaliquidaebit":{"Item1":null,"Item2":null},"dividaliquidapatrimonioliquido":{"Item1":null,"Item2":null},"p_sr":{"Item1":null,"Item2":null},"p_capitalgiro":{"Item1":null,"Item2":null},"p_ativocirculante":{"Item1":null,"Item2":null},"roe":{"Item1":null,"Item2":null},"roic":{"Item1":null,"Item2":null},"roa":{"Item1":null,"Item2":null},"liquidezcorrente":{"Item1":null,"Item2":null},"pl_ativo":{"Item1":null,"Item2":null},"passivo_ativo":{"Item1":null,"Item2":null},"giroativos":{"Item1":null,"Item2":null},"receitas_cagr5":{"Item1":null,"Item2":null},"lucros_cagr5":{"Item1":null,"Item2":null},"liquidezmediadiaria":{"Item1":null,"Item2":null},"vpa":{"Item1":null,"Item2":null},"lpa":{"Item1":null,"Item2":null},"valormercado":{"Item1":null,"Item2":null}}',
    "orderColumn": "",
    "isAsc": "",
    "page": 0,  # Começamos da página 0
    "take": 15,  # Número de itens por página
    "CategoryType": 1
}

# Cabeçalhos da solicitação
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Lista para armazenar os DataFrames de cada página
all_dfs = []

# Loop para iterar sobre todas as páginas
while True:
    response = requests.get(base_url, params=search_params, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['list'])
        all_dfs.append(df)
        
        # Incrementando o número da página
        search_params["page"] += 1
    else:
        print(f"Erro ao acessar a URL: {response.status_code}")
        break
    
    # Verificando se há mais páginas
    if not data['list']:
        break

# Concatenando todos os DataFrames em um único DataFrame
nova_base = pd.concat(all_dfs, ignore_index=True)

# Exibindo os resultados
print(nova_base.head())


   companyid                     companyname  ticker  price   p_l  p_vp   
0        205  FERROVIA CENTRO-ATLANTICA S.A.   VSPT4   0.00  0.00  0.00  \
1        226               VESTE S.A. ESTILO   VSTE3  14.10  6.76  1.56   
2        485          VULCABRAS/AZALEIA S.A.   VULC3  18.20  8.80  2.50   
3        216          GRUPO CASAS BAHIA S.A.  VVAR11  14.58 -0.78  0.31   
4        216          GRUPO CASAS BAHIA S.A.   VVAR4  95.68 -5.09  2.05   

   p_ebit  p_ativo  ev_ebit  margembruta  ...  valormercado  segmentid   
0    0.00     0.00    -1.03         7.78  ...  7.126108e+08         16  \
1    7.63     0.95     8.87        64.99  ...  1.607407e+09         22   
2    9.13     1.77     9.15        40.55  ...  4.968896e+09         32   
3   -8.30     0.04   -46.21        28.99  ...  7.045695e+08         20   
4  -54.48     0.28   -46.21        28.99  ...  7.045695e+08         20   

   sectorid  subsectorid                  subsectorname   
0       1.0          6.0                     

In [243]:
nova_base.fillna(0, inplace=True)
previsores_nova_base = nova_base.drop(['price', 'ticker', 'companyname', 'companyid', 'segmentid', 'sectorid', 'subsectorid', 'subsectorname', 'segmentname', 'sectorname'], axis=1)
for column in previsores_nova_base.columns:
    if column != 'ticker':
        previsores_nova_base[column] = previsores_nova_base[column].astype(str).apply(lambda x: x.replace('.', '').replace(',', '.') if ',' in x else x)
        previsores_nova_base[column] = previsores_nova_base[column].astype(float)


In [244]:
scaler = StandardScaler()
previsores_nova_scalonados = scaler.fit_transform(previsores_nova_base)

In [245]:
previsores_nova_base = pd.DataFrame(previsores_nova_base, columns=previsores_nova_base.columns)
previsores_nova_base['ticker'] = nova_base['ticker']
resultados_predicoes = previsores_nova_base[['ticker']].copy()
preco_real = nova_base['price']


In [246]:
for nome, _ in resultados_completos['resultados'].items():
    with open(f'{algoritmos_dir}/{nome}.pickle', 'rb') as file:
        modelo = pickle.load(file)
        resultados_predicoes[f'Preco_{nome}'] = modelo.predict(previsores_nova_scalonados)

In [247]:
resultados_predicoes['Preco_Real'] = preco_real
for nome in resultados_completos['resultados'].keys():
    resultados_predicoes[f'Dif_{nome}'] = abs(nova_base['price'] - resultados_predicoes['Preco_Real'])

In [248]:

# Identificar o modelo com o melhor R^2
melhor_modelo = max(resultados_completos['resultados'], key=lambda x: resultados_completos['resultados'][x]['r2'])
print(f"Melhor modelo: {melhor_modelo}")

# Criar um gráfico de barras verticais para o modelo
# Adicionando a coluna de diferenças para o melhor modelo
resultados_predicoes['Diferenca'] = abs(resultados_predicoes['Preco_Real'] - resultados_predicoes[f'Preco_{melhor_modelo}'])

# Ordenando pela maior diferença
resultados_predicoes.sort_values('Diferenca', ascending=False, inplace=True)

# Formatando valores em reais
resultados_predicoes['Preco_Real'] = resultados_predicoes['Preco_Real'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
resultados_predicoes[f'Preco_{melhor_modelo}'] = resultados_predicoes[f'Preco_{melhor_modelo}'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))
resultados_predicoes['Diferenca'] = resultados_predicoes['Diferenca'].apply(lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

# Criando a tabela com as colunas relevantes
tabela_diferencas = resultados_predicoes[['ticker', 'Preco_Real', f'Preco_{melhor_modelo}', 'Diferenca']]
tabela_diferencas.reset_index(drop=True, inplace=True)

# Exibindo a tabela
print(tabela_diferencas.to_string(index=False))

Melhor modelo: catboost
ticker   Preco_Real Preco_catboost Diferenca
 MSRO3    R$ 702,50      R$ 240,82 R$ 461,68
 MOSI3      R$ 8,24      R$ 112,89 R$ 104,65
 EEEL4    R$ 410,00      R$ 309,14 R$ 100,86
 AZUL4     R$ 13,32       R$ 86,84  R$ 73,52
 GUAR4    R$ 151,00       R$ 94,28  R$ 56,72
 BSEV3      R$ 9,00       R$ 58,90  R$ 49,90
 PRNR3     R$ 11,68       R$ 57,21  R$ 45,53
 PGMN3      R$ 2,80       R$ 46,21  R$ 43,41
 HAPV3      R$ 3,82       R$ 46,07  R$ 42,25
 CASN3      R$ 9,61       R$ 51,60  R$ 41,99
 SPRI3     R$ 21,72       R$ 61,61  R$ 39,89
 MNPR3     R$ 19,50       R$ 58,75  R$ 39,25
 GPAR3     R$ 42,77       R$ 78,94  R$ 36,17
 MNDL3     R$ 41,00       R$ 76,63  R$ 35,63
 CASN4      R$ 8,37       R$ 43,45  R$ 35,08
 SPRI6     R$ 11,00       R$ 45,51  R$ 34,51
 NINJ3      R$ 5,16       R$ 36,38  R$ 31,22
 SPRI5      R$ 8,52       R$ 39,73  R$ 31,21
 FNCN3     R$ 14,25       R$ 43,72  R$ 29,47
 ELET5     R$ 99,00       R$ 73,99  R$ 25,01
 ENMT3     R$ 80,00       R$ 55

In [310]:
tabela_diferencas.to_csv('./dados/nomes_acoes.csv')

In [301]:
combined_df.to_csv('./dados/dados_historicos.csv')